# Estimating Parameters with Spectral Methods

Now that we have some intuition for spectral methods, we will show how to use them to estimate parameters a little bit more rigorously. We will begin with the Random Dot Product Graph (RDPG), and in [Chapter 8](#link?), we will explore how to use the strategies we develop for the RDPG on a Stochastic Block Model where we do not know the node communities aheaad of time.

## Random Dot Product Graph

The Random Dot Product Graph has a single parameter, the latent position matrix $X$. Remember that the latent position matrix has $n$ rows, one for each node in the network, and $d$ columns, one for each latent dimension in the network. 

Remember that when we see a network which we think might be a realization of an RDPG random network, we do not actually have the latent position matrix $X$ ahead of time. For this reason, we must *estimate* $X$. Unfortunately, we cannot just use MLE like we did for the ER and SBM networks. Instead, we will use spectral methods.

In order to produce an estimate of $X$, we also need to know the number of latent dimensions of $\pmb A$, $d$. We might have a reasonable ability to "guess" what $d$ is ahead of time, but this will often not be the case. For this reason, we can instead estimate $d$ using the strategy described in [6.3.7](link?). Once we have an adjacency matrix $A$ and a number of latent dimensions $d$ (or an estimate of the number of latent dimensions, which we will call $\hat d$), producing an estimate of the latent position matrix is very straightforward. We simply use the adjacency spectral embedding to embed the adjacency matrix $A$ into $d$ (or, $\hat d$) dimensions. The resulting embedded adjacency matrix *is* the estimate of the latent position matrix. We will call the estimate of the latent position matrix $\hat X$.

Let's try an example of an *a priori* RDPG. We will use the same example that we used in the [section on RDPGs](#link?), where we had $100$ people living along a $100$ mile road, with each person one mile apart. The nodes represented people, and two people were connected if they were friends. The people at the ends of the street hosted parties, and invite everyone along the street to their parties. If a person was closer to one party host, they would tend to go to that host's parties more than the other party host. The latent positions for each person $i$ were the vectors:
\begin{align*}
    \vec x_i &= \begin{bmatrix}
        \frac{100 - i}{100} \\ \frac{i}{100}
    \end{bmatrix}
\end{align*}
In this case, since each $\vec x_i$ is $2$-dimensional, the number of latent dimensions in $X$ is $d=2$. Let's simulate an example network:

In [ ]:
import numpy as np
from graspologic.simulations import rdpg

n = 100  # the number of nodes in our network

# design the latent position matrix X according to 
# the rules we laid out previously
X = np.zeros((n,2))
for i in range(0, n):
    X[i,:] = [((n - i)/n), (i/n)]

P = X @ np.transpose(X)

np.random.seed(12)
A = rdpg(X)

In [ ]:
from graphbook_code import draw_multiplot
from graspologic.plot import heatmap
import matplotlib.pyplot as plt

draw_multiplot(A, title="Simulated RDPG(X)");

What happens when we fit a `rdpg` model to $A$? We will evaluate the performance of the RDPG estimator by comparing the estimated probability matrix, $\hat P = \hat X \hat X^\top$, to the true probability matrix, $P = XX^\top$. We can do this using the `RDPGEstimator` object, provided directly by graspologic:

In [ ]:
from graspologic.models import RDPGEstimator

model = RDPGEstimator(n_components=2, loops=False)  # number of latent dimensions is 2
model.fit(A)
Xhat = model.latent_
Phat = Xhat @ np.transpose(Xhat)

In [ ]:
from graphbook_code import plot_latents

fig, axs = plt.subplots(1, 2, figsize=(12, 6))

heatmap(Phat,
        vmin=0,
        vmax=1,
        font_scale=1.5,
        title="Estimated probability matrix",
        ax=axs[0])

heatmap(P,
        vmin=0,
        vmax=1,
        font_scale=1.5,
        title="True probability matrix",
        ax=axs[1])

fig;

Note that our estimated probability matrix tends to preserve the pattern in the true probability matrix, where the probabilities are highest for pairs of nodes which are closer together, but lower for pairs of nodes which are farther apart. 

What if we did not know that there were two latent dimensions ahead of time? The RDPG Estimator handles this situation just as well, and we can estimate the number of latent dimensions with $\hat d$ instead:

In [ ]:
model = RDPGEstimator(loops=False)  # number of latent dimensions is not known
model.fit(A)
Xhat = model.latent_
Phat = Xhat @ np.transpose(Xhat)
dhat = model.latent_.shape[1]
print("Estimated number of latent dimensions: {}".format(dhat))

So we can see that choosing the best-fit elbow instead yielded $\hat d = 3$; that is, the number of latent dimensions are estimated to be $3$. Again, looking at the estimated and true probability matrices:

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

heatmap(Phat,
        vmin=0,
        vmax=1,
        font_scale=1.5,
        title="Estimated probability matrix",
        ax=axs[0])

heatmap(P,
        vmin=0,
        vmax=1,
        font_scale=1.5,
        title="True probability matrix",
        ax=axs[1])

fig;

Which also is a decent estimate of the true probability matrix $P$.